In [4]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [5]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

userknn = UserKNNCFRecommender(URM_train)
itemknn = ItemKNNCFRecommender(URM_train)

UserKNNCFRecommender: URM Detected 236 ( 1.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 236 ( 1.9%) users with no interactions.
ItemKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.


In [ ]:
import numpy as np 
import scipy.sparse as sps
profile_length = np.ediff1d(sps.csr_matrix(URM_train).indptr)

#profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)
profile_length, profile_length.shape

In [6]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2176 (17.2%) Users that have less than 1 test interactions


In [ ]:
# PROFILE LENGTH BASED GROUPING
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

MAP_recommender_per_group = {}
cutoff = 10
threshold = 3

# Select users whose profile length is less than the cutoff for the group and greater than or equal to the cutoff for the previous group

users_in_group_flag = profile_length < threshold
users_in_group = np.arange(len(profile_length))[users_in_group_flag]

users_in_group_p_len = profile_length[users_in_group]

# Select users not in the current group
users_not_in_group_flag = np.isin(np.arange(len(profile_length)), users_in_group, invert=True)
users_not_in_group = np.arange(len(profile_length))[users_not_in_group_flag]

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[cutoff], ignore_users=users_not_in_group)


In [7]:
from Hybrid import TwoScoresHybridRecommender
import optuna as op
# COTRAINING
def objective(trial):
    ## USERKNN TRAINING
    user_topK = trial.suggest_int("user_topK", 2, 1e4)
    user_shrink = trial.suggest_int("user_shrink", 0, 1000)
    user_similarity = trial.suggest_categorical("user_similarity", ["cosine", "jaccard", "asymmetric", "dice", "tversky"])
    user_normalize = trial.suggest_categorical("user_normalize", [True, False])
    user_feature_weighting = trial.suggest_categorical("user_feature_weighting", ["none", "BM25", "TF-IDF"])
    userknn.fit(topK=user_topK, shrink=user_shrink, similarity=user_similarity, normalize=user_normalize, feature_weighting=user_feature_weighting)
    ## ITEMKNN TRAINING
    item_topK = trial.suggest_int("item_topK", 2, 10000)
    item_shrink = trial.suggest_int("item_shrink", 2, 1000)
    item_similarity = trial.suggest_categorical("item_similarity", ["cosine", "jaccard", "asymmetric", "dice", "tversky"])
    item_normalize = trial.suggest_categorical("item_normalize", [True, False])
    item_feature_weighting = trial.suggest_categorical("item_feature_weighting", ["none", "BM25", "TF-IDF"])
    itemknn.fit(topK=item_topK, shrink=item_shrink, similarity=item_similarity, normalize=item_normalize, feature_weighting=item_feature_weighting)
    
    ## HYBRID
    
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    
    recommender = TwoScoresHybridRecommender(URM_train, userknn, itemknn)
    recommender.fit(alpha=alpha)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [13]:
study_name = "KNN-cotrain-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.optimize(objective, n_trials=100)

[I 2023-12-22 11:18:26,121] Using an existing study with name 'KNN-cotrain-study' instead of creating a new one.


Similarity column 12638 (100.0%), 12329.77 column/sec. Elapsed time 1.03 sec
Similarity column 22222 (100.0%), 10166.53 column/sec. Elapsed time 2.19 sec
TwoScoresHybridRecommender: URM Detected 236 ( 1.9%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 115 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 10462 (100.0%) in 8.51 sec. Users per second: 1229


[I 2023-12-22 11:18:39,031] Trial 500 finished with value: 0.03112420460441899 and parameters: {'user_topK': 6595, 'user_shrink': 893, 'user_similarity': 'cosine', 'user_normalize': True, 'user_feature_weighting': 'none', 'item_topK': 6, 'item_shrink': 81, 'item_similarity': 'asymmetric', 'item_normalize': True, 'item_feature_weighting': 'TF-IDF', 'alpha': 0.6438562299363502}. Best is trial 215 with value: 0.048046896098655506.


Similarity column 12638 (100.0%), 12225.70 column/sec. Elapsed time 1.03 sec
Similarity column 22222 (100.0%), 12812.00 column/sec. Elapsed time 1.73 sec
TwoScoresHybridRecommender: URM Detected 236 ( 1.9%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 115 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 10462 (100.0%) in 8.45 sec. Users per second: 1238


[I 2023-12-22 11:18:50,980] Trial 501 finished with value: 0.029227813887296106 and parameters: {'user_topK': 7809, 'user_shrink': 964, 'user_similarity': 'cosine', 'user_normalize': True, 'user_feature_weighting': 'none', 'item_topK': 2, 'item_shrink': 138, 'item_similarity': 'jaccard', 'item_normalize': True, 'item_feature_weighting': 'TF-IDF', 'alpha': 0.7239882338900044}. Best is trial 215 with value: 0.048046896098655506.


...

[I 2023-12-22 11:41:23,447] Trial 597 finished with value: 0.042609109915552765 and parameters: {'user_topK': 9961, 'user_shrink': 961, 'user_similarity': 'cosine', 'user_normalize': True, 'user_feature_weighting': 'none', 'item_topK': 249, 'item_shrink': 447, 'item_similarity': 'asymmetric', 'item_normalize': True, 'item_feature_weighting': 'TF-IDF', 'alpha': 0.7396698682540004}. Best is trial 215 with value: 0.048046896098655506.


Similarity column 12638 (100.0%), 11971.82 column/sec. Elapsed time 1.06 sec
Similarity column 22222 (100.0%), 10050.64 column/sec. Elapsed time 2.21 sec
TwoScoresHybridRecommender: URM Detected 236 ( 1.9%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 115 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 10462 (100.0%) in 11.50 sec. Users per second: 910


[I 2023-12-22 11:41:39,340] Trial 598 finished with value: 0.04259264822350307 and parameters: {'user_topK': 8578, 'user_shrink': 875, 'user_similarity': 'cosine', 'user_normalize': True, 'user_feature_weighting': 'none', 'item_topK': 749, 'item_shrink': 368, 'item_similarity': 'asymmetric', 'item_normalize': True, 'item_feature_weighting': 'TF-IDF', 'alpha': 0.656415163765803}. Best is trial 215 with value: 0.048046896098655506.


Similarity column 12638 (100.0%), 12344.57 column/sec. Elapsed time 1.02 sec
Similarity column 22222 (100.0%), 10306.93 column/sec. Elapsed time 2.16 sec
TwoScoresHybridRecommender: URM Detected 236 ( 1.9%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 115 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 10462 (100.0%) in 10.89 sec. Users per second: 960


[I 2023-12-22 11:41:54,255] Trial 599 finished with value: 0.03916170919396935 and parameters: {'user_topK': 8084, 'user_shrink': 928, 'user_similarity': 'cosine', 'user_normalize': True, 'user_feature_weighting': 'none', 'item_topK': 181, 'item_shrink': 803, 'item_similarity': 'jaccard', 'item_normalize': True, 'item_feature_weighting': 'TF-IDF', 'alpha': 0.8314796073819051}. Best is trial 215 with value: 0.048046896098655506.
